In [ ]:
print("\n... IMPORTS STARTING ...\n")

import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
print("\n\tVERSION INFORMATION")
# Machine Learning and Data Science Imports
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");""
import logging

#import tensorflow_hub as tfhub; print(f"\t\t– TENSORFLOW HUB VERSION: {tfhub.__version__}");
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");
import sklearn; print(f"\t\t– SKLEARN VERSION: {sklearn.__version__}");
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from pandarallel import pandarallel; pandarallel.initialize();
from sklearn.model_selection import GroupKFold, StratifiedKFold, train_test_split

# # RAPIDS
# import cudf, cupy, cuml
# from cuml.neighbors import NearestNeighbors
# from cuml.manifold import TSNE, UMAP

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
from glob import glob
#import warnings
import requests
import hashlib
import IPython
import sklearn
import urllib
import zipfile
import pickle
import random
import shutil
import string
import json
import math
import time
import gzip
import ast
import sys
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import seaborn as sns
from PIL import Image, ImageEnhance
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
from matplotlib import animation, rc; rc('animation', html='jshtml')
import plotly
import PIL
import cv2

import plotly.io as pio
print(pio.renderers)

def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
tf.get_logger().setLevel('ERROR')
    
print("\n\n... IMPORTS COMPLETE ...\n")

print("\n... SEEDING FOR DETERMINISTIC BEHAVIOUR ...\n")
seed_it_all()

In [ ]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

In [ ]:
print(f"\n... ACCELERATOR SETUP STARTING ...\n")

# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  
except ValueError:
    TPU = None

if TPU:
    print(f"\n... RUNNING ON TPU - {TPU.master()}...")
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.experimental.TPUStrategy(TPU)
else:
    print(f"\n... RUNNING ON CPU/GPU ...")
    physical_devices = tf.config.list_physical_devices('GPU')
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except:
        # Invalid device or cannot modify virtual devices once initialized.
        pass
    
    # Yield the default distribution strategy in Tensorflow
    #   --> Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

# What Is a Replica?
#    --> A single Cloud TPU device consists of FOUR chips, each of which has TWO TPU cores. 
#    --> Therefore, for efficient utilization of Cloud TPU, a program should make use of each of the EIGHT (4x2) cores. 
#    --> Each replica is essentially a copy of the training graph that is run on each core and 
#        trains a mini-batch containing 1/8th of the overall batch size
N_REPLICAS = strategy.num_replicas_in_sync
    
print(f"... # OF REPLICAS: {N_REPLICAS} ...\n")

print(f"\n... ACCELERATOR SETUP COMPLTED ...\n")

In [ ]:
print("\n... DATA ACCESS SETUP STARTED ...\n")

if TPU:
    # Google Cloud Dataset path to training and validation images
    DATA_DIR = KaggleDatasets().get_gcs_path('uw-madison-gi-tract-image-segmentation')
    save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
    load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
else:
    # Local path to training and validation images
    DATA_DIR = "../input/uw-madison-gi-tract-image-segmentation"
    save_locally = None
    load_locally = None

print(f"\n... DATA DIRECTORY PATH IS:\n\t--> {DATA_DIR}")

print(f"\n... IMMEDIATE CONTENTS OF DATA DIRECTORY IS:")
for file in tf.io.gfile.glob(os.path.join(DATA_DIR, "*")): print(f"\t--> {file}")

print("\n\n... DATA ACCESS SETUP COMPLETED ...\n")
#'gs://kds-5cfbc058b17caa8a1bd8982b1193f1c5381f45c742948f2a940c250d/train/**/*.png'

In [ ]:
print(f"\n... XLA OPTIMIZATIONS STARTING ...\n")

print(f"\n... CONFIGURE JIT (JUST IN TIME) COMPILATION ...\n")
# enable XLA optmizations (10% speedup when using @tf.function calls)
tf.config.optimizer.set_jit(False)

print(f"\n... XLA OPTIMIZATIONS COMPLETED ...\n")

In [ ]:
print("\n... Basic Data Set up Starting ... ")
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TRAIN_CSV_DIR = os.path.join(DATA_DIR, "train.csv")
train_df = pd.read_csv(TRAIN_CSV_DIR)
#all_training_images = tf.io.gfile.glob(os.path.join(TRAIN_DIR, "**", "*.png"))
print("\n... ORIGINAL TRAINING DATAFRAME... \n")
display(train_df)

# Test cases and submission csv check
TEST_DIR = os.path.join(DATA_DIR, "test")
print(TEST_DIR)
SS_CSV   = os.path.join(DATA_DIR, "sample_submission.csv")
ss_df = pd.read_csv(SS_CSV)

all_test_images = glob(os.path.join(TEST_DIR, "**", "*.png"), recursive=True)
print("Test images in total: ", len(all_test_images))
print("\n\n\n... ORIGINAL SUBMISSION DATAFRAME... \n")
display(ss_df)

In [ ]:
def get_all_training_images(df, is_tf = True):
    if(is_tf):
        df["case_#"] = df["id"].apply(lambda x : x.split("_")[0])
        df["day_#"] = df["id"].apply(lambda x : x.split("_")[1])
        df_tf = pd.DataFrame({"case_#" : df["case_#"], "day_#":df["day_#"]})
        df_tf = df_tf.drop_duplicates(subset=["case_#", "day_#"]).reset_index(drop=True)
        df_tf["case#_day#"] = df["case_#"]+"_"+df["day_#"]
        all_image_path = df_tf["case#_day#"].apply(lambda x : tf.io.gfile.glob(os.path.join(TRAIN_DIR, x.split("_")[0], x, "scans", "*.png")))
        all_train_image_path = []
        for paths in all_image_path:
            all_train_image_path = all_train_image_path + paths
        
    else:
        all_train_image_path = glob(os.path.join(TRAIN_DIR, "**", "*.png"), recursive=True)
    return all_train_image_path

all_training_images = get_all_training_images(train_df.copy(), is_tf = False)

In [ ]:
print("...There are in total : ", len(all_training_images), " images")
print("...One of Image file paths :  \n", all_training_images[10000])

In [ ]:
# For debugging purposes when the test set hasn't been substituted we will know
DEBUG=len(all_test_images)==0



if DEBUG:
    TEST_DIR = TRAIN_DIR
    all_test_images = all_training_images
    first_50_cases = train_df.id.apply(lambda x: x.split("_", 1)[0]).unique()[:50]
    ss_df = train_df[train_df.id.apply(lambda x: x.split("_", 1)[0]).isin(first_50_cases)]
    ss_df = ss_df[["id", "class"]]
    ss_df["predicted"] = ""
    
    print("\n\n\n... DEBUG SUBMISSION DATAFRAME... \n")
    display(ss_df)

In [ ]:
classes = ["Large Bowel", "Small Bowel", "Stomach"]
sf_classes = ["lb", "sb", "st"]
SF2LF = {_sf:_lf for _sf,_lf in zip(sf_classes, classes)}
LF2SF = {_lf:_sf for _sf,_lf in zip(sf_classes, classes)}

In [ ]:
def separateKeyWords(df):
    df["case_id_str"] = df["id"].apply(lambda x : x.split("_")[0])
    df["case_id"] = df["case_id_str"].apply(lambda x : int(x[4:]))
    df["day_num_str"] = df["id"].apply(lambda x : x.split("_")[1])
    df["slice_id_str"] = df["id"].apply(lambda x : x.split("_",2)[-1])
    return df

def set_up_img_file_path(df, img_files_paths):
    # correct path: '../input/uw-madison-gi-tract-image-segmentation/train/case36/case36_day14/scans/slice_0006_266_266_1.50_1.50.png'
    #gs://kds-5cfbc058b17caa8a1bd8982b1193f1c5381f45c742948f2a940c250d/train/case123/case123_day20/scans/slice_0001_266_266_1.50_1.50.png
    identifier = [x.rsplit("/",3)[1] +"_"+ x.rsplit("/",1)[1].rsplit("_", 4)[0] for x in img_files_paths]
    _tmp_merge_df = pd.DataFrame({"id" : identifier, "img_file_path" : img_files_paths})
    #display(_tmp_merge_df)
    df = df.merge(_tmp_merge_df, on = "id")
    return df

def process_segmentation(df):
    # Merge three rows which corresponding to three types of segmentations into one row
    large_bowel_df = df[df["class"] == "large_bowel"][["id", "segmentation"]].rename(columns = {"segmentation" : "large_bowel_seg_rle"})
    small_bowel_df = df[df["class"] == "small_bowel"][["id", "segmentation"]].rename(columns = {"segmentation" : "small_bowel_seg_rle"})
    stomach_df = df[df["class"] == "stomach"][["id", "segmentation"]].rename(columns = {"segmentation" : "stomach_seg_rle"})
    df = df.merge(large_bowel_df, on = "id", how = "left")
    df = df.merge(small_bowel_df, on = "id", how = "left")
    df = df.merge(stomach_df, on = "id", how = "left")
    df = df.drop_duplicates(subset=["id",]).reset_index(drop=True)
    df["large_bowel_seg_flag"] = df["large_bowel_seg_rle"].apply(lambda x: not pd.isna(x))
    df["small_bowel_seg_flag"] = df["small_bowel_seg_rle"].apply(lambda x: not pd.isna(x))
    df["stomach_seg_flag"] = df["stomach_seg_rle"].apply(lambda x: not pd.isna(x))
    df.drop(["class","segmentation"], axis = 1, inplace = True)
    return df

def get_height_and_width(df):
    df["slice_h"] = df["img_file_path"].apply(lambda x : int(x[:-4].rsplit("_",4)[1]))
    df["slice_w"] = df["img_file_path"].apply(lambda x : int(x[:-4].rsplit("_",4)[2]))
    df["px_spacing_h"] = df["img_file_path"].apply(lambda x : float(x[:-4].rsplit("_",4)[3]))
    df["px_spacing_w"] = df["img_file_path"].apply(lambda x : float(x[:-4].rsplit("_",4)[4]))
    return df

In [ ]:
def data_first_stage_preprocessing(df, img_files_paths, is_submitting = False):
# 276_276_1.63_1.63.png.
#These four numbers are slice height / width (integers in pixels) and heigh/width pixel spacing (floating points in mm).
#The first two: the resolution of the slide. The last two: the physical size of each pixel.
    df = separateKeyWords(df)
    if not is_submitting:
        df = process_segmentation(df)
    df = set_up_img_file_path(df, img_files_paths)
    df = get_height_and_width(df)
    # 8. Reorder columns to the a new ordering (drops class and segmentation as no longer necessary)
    new_col_order = ["id", "img_file_path",
                     "large_bowel_seg_rle", "large_bowel_seg_flag",
                     "small_bowel_seg_rle", "small_bowel_seg_flag", 
                     "stomach_seg_rle", "stomach_seg_flag",
                     "slice_h", "slice_w", "px_spacing_h", 
                     "px_spacing_w", "case_id_str", "case_id", 
                     "day_num_str", "day_num", "slice_id", "predicted"]
    if is_submitting: new_col_order.insert(1, "class")
    new_col_order = [_c for _c in new_col_order if _c in df.columns]
    df = df[new_col_order]
    return df
    

In [ ]:
def data_second_stage_preprocessing(train_df):
    N_FOLDS = 8
    gkf = GroupKFold(n_splits=N_FOLDS) 
    
    remove_ids = ["case7_day0", "case81_day30"]
    for _id in remove_ids:
        train_df = train_df[~train_df.id.str.contains(_id)].reset_index(drop=True)
    
    # train_df = train_df[train_df.n_segs>0].reset_index(drop=True)
    train_df["which_segs"] = train_df.large_bowel_seg_flag.astype(int).astype(str)+\
                             train_df.small_bowel_seg_flag.astype(int).astype(str)+\
                             train_df.stomach_seg_flag.astype(int).astype(str)
    
    """
    Test train val split
    Total 38496
    
    Get train 60% = 26,947
        val 20% = 5774
        test 20% = 5774
    """
    train_df, test_df = np.split(train_df.sample(frac=1), [int(.9*len(train_df))])
    
    for train_idxs, val_idxs in gkf.split(train_df["id"], train_df["which_segs"], train_df["case_id"]):
        sub_train_df=train_df.iloc[train_idxs]
        N_TRAIN = len(sub_train_df)
        sub_train_df=sub_train_df.sample(N_TRAIN).reset_index(drop=True)

        sub_val_df=train_df.iloc[val_idxs]
        N_VAL = len(sub_val_df)
        sub_val_df=sub_val_df.sample(N_VAL).reset_index(drop=True)

        break

    # Fix the way we handled nan
    sub_train_df.large_bowel_seg_rle.fillna("", inplace=True)
    sub_train_df.small_bowel_seg_rle.fillna("", inplace=True)
    sub_train_df.stomach_seg_rle.fillna("", inplace=True)
    
    # Fix the way we handled nan
    sub_val_df.large_bowel_seg_rle.fillna("", inplace=True)
    sub_val_df.small_bowel_seg_rle.fillna("", inplace=True)
    sub_val_df.stomach_seg_rle.fillna("", inplace=True)
    
    test_df.large_bowel_seg_rle.fillna("", inplace=True)
    test_df.small_bowel_seg_rle.fillna("", inplace=True)
    test_df.stomach_seg_rle.fillna("", inplace=True)

    print("\nFOLD 1: TRAIN DF\n\n")
    display(sub_train_df.head())

    print("\n\n\n\nFOLD 1: VAL DF\n\n")
    display(sub_val_df.head())
    
    print("\n\n\n TEST DF\n\n")
    display(test_df.head())
    
    return sub_train_df, sub_val_df, test_df

In [ ]:
train_df_after_first_stage = data_first_stage_preprocessing(train_df.copy(), all_training_images)
ss_df = data_first_stage_preprocessing(ss_df, all_test_images, is_submitting =True)
display(ss_df)
display(train_df_after_first_stage.head())

In [ ]:
train_df_preprocessed, val_df, test_df = data_second_stage_preprocessing(train_df_after_first_stage)

In [ ]:
total = len(train_df_after_first_stage)
print("\n ...Training set ratio to the total examples :    ", "%.2f"%(len(train_df_preprocessed)/total),"\n")
print("\n ...Validation set ratio to the total examples :    ", "%.2f"%(len(val_df)/total), "\n")
print("\n ...Test set ratio to the total examples :    ", "%.2f"%(len(test_df)/total), "\n")

In [ ]:
"""
TF Functions:
1 load mask as a tensorflow object
1 for decode rln
"""

IMAGE_SHAPE = SEG_SHAPE = (256,256)
def rle_encode(img):
    """ TBD
    
    Args:
        img (np.array): 
            - 1 indicating mask
            - 0 indicating background
    
    Returns: 
        run length as string formated
    """
    
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def tf_load_mask(rle_strs, root_shape, style="multiclass"):
    """ 
    Pure tensorflow function to load multiple rles into an RGB array based on output style 
    For now, I am doing multiclass 
    """
    tf_masks = [tf.cast(tf.image.resize(tf.expand_dims(rle_decode_tf(rle_str, root_shape), axis=-1), \
                                        size=(tf.constant(SEG_SHAPE[0]), tf.constant(SEG_SHAPE[1])), \
                                        method=tf.image.ResizeMethod.NEAREST_NEIGHBOR), tf.uint8) for rle_str in rle_strs]
    if style=="multiclass": # stack for each class
        return tf.concat(tf_masks, axis=-1)
    else:        
        _tf_masks = tf.zeros((*SEG_SHAPE, 1), dtype=tf.uint8)
        _tf_masks = tf_masks[2]*tf.constant(3, dtype=tf.uint8) # small bowel = 3
        _tf_masks = tf.where(tf_masks[1]==tf.constant(1, dtype=tf.uint8), tf.constant(2, dtype=tf.uint8), _tf_masks) # small bowel = 2
        _tf_masks = tf.where(tf_masks[0]==tf.constant(1, dtype=tf.uint8), tf.constant(1, dtype=tf.uint8), _tf_masks) # large bowel = 1
        return _tf_masks
    
    
def tf_load_image(path):
    """ Load an image with the correct shape using only TF
    
    Args:
        path (tf.string): Path to the image to be loaded
        resize_to (tuple, optional): Size to reshape image
    
    Returns:
        3 channel tf.Constant image ready for training/inference
    
    """
    img_bytes = tf.io.read_file(path)
    img = tf.image.decode_png(img_bytes, channels=3, dtype=tf.uint16)
    # img = 255.*(img/tf.constant(32767, dtype=tf.uint16))
    img = 255.*(img/tf.reduce_max(img)) # Normalization
    img = tf.image.resize(img, (tf.constant(IMAGE_SHAPE[0]), tf.constant(IMAGE_SHAPE[1])))
    return img
# Try show the an image with the original function and with normalization with tensorflow function


def rle_decode_tf(mask_rle, shape):
    """ Pure tensorflow RLE decoding function for easy pipelining
    
    Args:
        mask_rle (str): The Run Length Encoded mask
        shape (tuple): The shape of the mask we are decoding
    
    Returns:
        A 1D tf.constant representing the decoded mask
        If mask_rle == "", return zeros tf.constant with shape
    """
    
    shape = tf.convert_to_tensor(shape, tf.int64)
    size = tf.math.reduce_prod(shape)
    
    # Split string
    s = tf.strings.split(mask_rle)
    s = tf.strings.to_number(s, tf.int64)
    
    # Get starts and lengths
    starts = s[::2] - 1
    lens = s[1::2]
    
    # Make ones to be scattered
    total_ones = tf.reduce_sum(lens)
    ones = tf.ones([total_ones], tf.uint8)
    
    # Make scattering indices
    r = tf.range(total_ones)
    lens_cum = tf.math.cumsum(lens)
    s = tf.searchsorted(lens_cum, r, 'right')
    idx = r + tf.gather(starts - tf.pad(lens_cum[:-1], [(1, 0)]), s)
    
    # Scatter ones into flattened mask
    mask_flat = tf.scatter_nd(tf.expand_dims(idx, 1), ones, [size])
    
    # Reshape into mask
    return tf.reshape(mask_flat, shape)
    #return tf.transpose(tf.reshape(mask_flat,(shape[1],shape[0])))

In [ ]:
"""
Visualization tools
"""
def showRandomSlices(num):
    row = (num // 5) + 1
    col = 5
    n = np.random.randint(low=0, high=len(all_training_images), size=num)
    plt.figure(figsize = (20,10))
    for i in range(num):
        plt.subplot(row,col,i+1)
        img = cv2.imread(all_training_images[n[i]],0)
        img = (img/img.max()).astype(np.float32) 
        #cv2.normalize(img,  img, 0, 255, cv2.NORM_MINMAX)
        #tf_img = tf.constant(img)
        #img = tf.image.per_image_standardization(tf_img)
        plt.imshow(img)
        plt.axis(False)
    plt.tight_layout()
    plt.show()


def get_overlay(img, mask, _alpha=0.999, _beta=0.45, _gamma=0):
    
    # Normalize to be between 0-1 (float32)
    img = (img/img.max()).astype(np.float32)
    
    # Make mask RGB and float32
    if len(mask.shape)==2:
        mask_rgb = np.zeros_like(img, dtype=np.float32)
        mask_rgb[..., 2] = np.where(mask==3, 1.0, 0.0)
        mask_rgb[..., 1] = np.where(mask==2, 1.0, 0.0)
        mask_rgb[..., 0] = np.where(mask==1, 1.0, 0.0)
    else:
        mask_rgb=mask.astype(np.float32)
    
    # overlay
    seg_overlay = cv2.addWeighted(src1=img, alpha=_alpha, 
                                  src2=mask_rgb, beta=_beta, gamma=_gamma)
    return seg_overlay



def plot_image_with_mask(img, gt_mask): #pred_mask
    """
    To visualize results after training
    * need modification
    """
    gt_overlay = get_overlay(img, gt_mask)
    #pred_overlay = get_overlay(img, pred_mask)
    #miss_overlay = get_miss_overlay(gt_mask, pred_mask)
    
    plt.figure(figsize=(10,6))
    #["Original", "Prediction Mask", "Ground-Truth Mask", "Miss Mask"]
    descriptions = ["Ground-Truth Mask"]
    
    for i, (_desc, _img) in enumerate(zip(descriptions, [gt_overlay,])):        
        plt.subplot(1,2,i+1)
        plt.imshow(_img)
        plt.title(f"{_desc} Image", fontweight="bold")        
        plt.axis(False)
        
        if i in [1,2]:
            handles = [Rectangle((0,0),1,1, color=_c) for _c in [(0.667,0.0,0.0), (0.0,0.667,0.0), (0.0,0.0,0.667)]]
            labels = ["Large Bowel Segmentation Map", "Small Bowel Segmentation Map", "Stomach Segmentation Map"]
            plt.legend(handles,labels)
        """ 
        elif i==3:
            handles = [Rectangle((0,0),1,1, color=_c) for _c in [(0.0,0.8,0.0), (0.8,0.0,0.0), (0.0, 0.0, 0.0)]]
            labels = ["Agreement", "Disagreement", "Background"]
            plt.legend(handles,labels)
        """
    plt.tight_layout()
    plt.show()

In [ ]:
"""
Visualization:
"""
if DEBUG:
    tdf = train_df_preprocessed
    all_labeled = tdf[tdf["which_segs"] == "111"].iloc[:4]
    
    all_labeled_ds = tf.data.Dataset.from_tensor_slices( (all_labeled["img_file_path"],\
                                                 (all_labeled.large_bowel_seg_rle, all_labeled.small_bowel_seg_rle, all_labeled.stomach_seg_rle),\
                                                 (all_labeled.slice_w,all_labeled.slice_h) ) )
    all_labeled_ds = all_labeled_ds.map(lambda x,y,z: (tf_load_image(x),tf.cast(tf_load_mask(y,z, style = "multiclass"), tf.float32 )))
    plt.figure(figsize=(15,9))
    i = 1
    for img, mask in all_labeled_ds:
        _mask = mask.numpy().squeeze().astype(np.float32)
        gt_overlay = get_overlay(img.numpy().astype(np.int32), _mask)
        plt.subplot(2,2,i)
        i+=1
        plt.imshow(gt_overlay)
        handles = [Rectangle((0,0),1,1, color=_c) for _c in [(0.667,0.0,0.0), (0.0,0.667,0.0), (0.0,0.0,0.667)]]
        labels = ["Large Bowel Segmentation Map", "Small Bowel Segmentation Map", "Stomach Segmentation Map"]
        plt.legend(handles,labels)
        plt.axis(False)
    plt.tight_layout()
    plt.show()

In [ ]:
def augment_batch(img_batch, mask_batch):
    """ Pipeline augmentation 
        - Right-Left Flipping (1/3 probability)
        - Rotation (2/3 probability)
        - Translation (2/3 probability)
    """
    # Simple augmentation
    if tf.random.uniform([])<=tf.constant(0.3333):
        img_batch = tf.image.flip_left_right(img_batch)
        mask_batch = tf.image.flip_left_right(mask_batch)
    # Random rotation
    if tf.random.uniform([])<=tf.constant(0.6666):
        degree_rot = tf.random.uniform([BATCH_SIZE,], minval=tf.constant(-0.4), maxval=tf.constant(0.4))
        img_batch = tfa.image.rotate(img_batch, degree_rot, interpolation="bilinear")
        mask_batch = tfa.image.rotate(mask_batch, degree_rot, interpolation="nearest")  
    # Random translation
    if tf.random.uniform([])<=tf.constant(0.6666):
        _t_mag = tf.random.uniform([BATCH_SIZE,2], minval=tf.constant(-30.0), maxval=tf.constant(30.0))
        img_batch = tfa.image.translate(img_batch, translations=_t_mag, interpolation="bilinear")
        mask_batch = tfa.image.translate(mask_batch, translations=_t_mag, interpolation="nearest")
        
    return img_batch, mask_batch

def add_sample_weights(image_batch, mask_batch, _multiplier=1.5, _exp=0.25):
    """
    Incorporate class weighting as a third term in tf.data.Dataset
        
        BACKGROUND TRAINING DATA PIXEL COUNT (%)  : %98.3326
        LARGE BOWEL TRAINING DATA PIXEL COUNT (%) : %0.6883
        SMALL BOWEL TRAINING DATA PIXEL COUNT (%) : %0.6301
        STOMACH TRAINING DATA PIXEL COUNT (%)     : %0.3490

    """    
    # Add class weighting
    likelihood = tf.constant([0.983326, 0.06883, 0.06301, 0.03490])
    class_weights = tf.constant(_multiplier)*((tf.constant(1.0)-likelihood)**_exp)
    
    # Create an image of `sample_weights` by using the label at each pixel as an index into the `class weights`
    #sample_weights_batch = tf.gather(class_weights, indices=tf.cast(mask_batch, tf.int32))
    
    """
    for each channel set up sample weight
    """
    height, width, channels = mask.shape
    output_mask = tf.zeros((height, width, 1))
    for c in range(channels): # shoudl be [0:3]
        _class_weights = [class_weights[0], class_weights[c+1]]
        output_mask = tf.concat([output_mask, tf.expand_dims(tf.gather(_class_weights, indices = tf.cast(mask[...,c], tf.int32) ), -1)], -1)
    sample_weights_batch = output_mask[...,1:]
    return image_batch, mask_batch, sample_weights_batch

def model_preprocessing_train(img_batch, mask_batch):
    """ Model specific preprocessing for DeepLabV3 (training)"""
    img_batch = img_batch/tf.constant(127.5)-tf.constant(1.0)
    return img_batch, mask_batch
    
def model_preprocessing_test(img_batch):
    """ Model specific preprocessing for DeepLabV3 (testing)"""    
    img_batch = img_batch/tf.constant(127.5)-tf.constant(1.0)
    return img_batch

def convert_2_float32_train(img_batch, mask_batch, sample_weights_batch):
    return tf.cast(img_batch, tf.float32), tf.cast(mask_batch, tf.float32), tf.cast(sample_weights_batch, tf.float32)

def convert_2_float32_val(img_batch, mask_batch):
    return tf.cast(img_batch, tf.float32), tf.cast(mask_batch, tf.float32)

# Hyperparameters
BATCH_SIZE = 24
SHUFFLE_BUFFER = max(BATCH_SIZE*25, 500)
AUTOTUNE = tf.data.AUTOTUNE

# Whether or not to train?

DO_TRAIN= False
STYLE = "multiclass"
AUTOTUNE = tf.data.AUTOTUNE
if not DEBUG: DO_TRAIN=False
    # DEBUG    -> True: on my own; False : submit
    # DO_TRAIN -> 
    
# train_df_preprocessed : the dataframe that has been cleaned and preprocessed
# val_df : same structure as the training df but different examples
    
if DEBUG:
    train_ds = tf.data.Dataset.from_tensor_slices((train_df_preprocessed.img_file_path,\
                                                   (train_df_preprocessed.large_bowel_seg_rle, train_df_preprocessed.small_bowel_seg_rle, train_df_preprocessed.stomach_seg_rle),\
                                                   (train_df_preprocessed.slice_w,train_df_preprocessed.slice_h)))
    
    val_ds = tf.data.Dataset.from_tensor_slices((val_df.img_file_path,\
                                                 (val_df.large_bowel_seg_rle, val_df.small_bowel_seg_rle, val_df.stomach_seg_rle),\
                                                 (val_df.slice_w,val_df.slice_h)))

    
    # Images are normalized within "tf_load_image"
    # Masks are created as stacks of multiple classes -> "multilabel"
    train_ds = train_ds.map(lambda x,y,z: (tf_load_image(x), tf_load_mask(y,z,style=STYLE)), num_parallel_calls=AUTOTUNE)
    
    val_ds = val_ds.map(lambda x,y,z: (tf_load_image(x), tf_load_mask(y,z,style=STYLE)), num_parallel_calls=AUTOTUNE)

    train_ds = train_ds.shuffle(SHUFFLE_BUFFER)\
                       .batch(BATCH_SIZE, drop_remainder=True)\
                       .map(augment_batch, num_parallel_calls=AUTOTUNE)\
                       .map(model_preprocessing_train, num_parallel_calls=AUTOTUNE)\
                       .map(add_sample_weights, num_parallel_calls=AUTOTUNE)\
                       .map(convert_2_float32_train)\
                       .prefetch(AUTOTUNE)    
    # .map(model_preprocessing_train, num_parallel_calls=AUTOTUNE)\
    
    # we only shuffle the validation a little because we don't want 
    # drop_remainder to hit the same images over and over...
    val_ds = val_ds.shuffle(SHUFFLE_BUFFER//5)\
                   .batch(BATCH_SIZE, drop_remainder=True)\
                   .map(model_preprocessing_train, num_parallel_calls=AUTOTUNE)\
                   .map(convert_2_float32_val)\
                   .prefetch(AUTOTUNE)
    
    #.map(model_preprocessing_train, num_parallel_calls=AUTOTUNE)\
    
    
    """
    for _img_batch, _mask_batch in val_ds.take(1):
        print(_img_batch.shape, _mask_batch.shape)
        _img=_img_batch[0]
        _mask=_mask_batch[0]
        plt.figure(figsize=(15,5))
        plt.subplot(1,2,1)
        plt.imshow(tf.cast(_mask, tf.float32))

        plt.subplot(1,2,2)
        plt.imshow(tf.cast((_img+1)*127.5, tf.uint8))

        plt.tight_layout()
        plt.show()"""
    

In [ ]:
test_ds = tf.data.Dataset.from_tensor_slices((test_df.img_file_path,\
                                                 (test_df.large_bowel_seg_rle, test_df.small_bowel_seg_rle, test_df.stomach_seg_rle),\
                                                 (test_df.slice_w, test_df.slice_h)))
test_ds = test_ds.map(lambda x,y,z: (tf_load_image(x), tf_load_mask(y,z,style=STYLE)), num_parallel_calls=AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE)\
                 .map(model_preprocessing_train, num_parallel_calls=AUTOTUNE)\
                 .prefetch(AUTOTUNE)

In [ ]:
print("Train Dataset:", train_ds)
print("Val Dataset:", val_ds)

In [ ]:
# We only need every third row (hence the iloc[::3])
submission_ds = tf.data.Dataset.from_tensor_slices(ss_df.iloc[::3].img_file_path.tolist())
submission_ds = submission_ds.map(lambda x: tf_load_image(x), num_parallel_calls=AUTOTUNE)
# This should be deterministic... i.e. the order of images will match the order of IDs
submission_ds = submission_ds.batch(BATCH_SIZE)\
                    .map(model_preprocessing_test, num_parallel_calls=AUTOTUNE)\
                    .prefetch(AUTOTUNE)

# ***Model Building***

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D, UpSampling2D, Reshape, Dense
from tensorflow.keras.models import Model

def SqueezeAndExcite(inputs, ratio=8):
    init = inputs
    filters = init.shape[-1]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    x = init * se
    return x

def ASPP(aspp_input):
    """
    Atrous(Dilated) pyramid pooling
    """
    shape = aspp_input.shape
    #Image Pooling
    y1 = AveragePooling2D(pool_size = (shape[1], shape[2]))(aspp_input)
    y1 = Conv2D(256, 1, padding="same", use_bias=False)(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y1)

    """ 1x1 conv """
    y2 = Conv2D(256, 1, padding="same", use_bias=False)(aspp_input)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    """ 3x3 conv rate=6 """
    y3 = Conv2D(256, 3, padding="same", use_bias=False, dilation_rate=6)(aspp_input)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    """ 3x3 conv rate=12 """
    y4 = Conv2D(256, 3, padding="same", use_bias=False, dilation_rate=12)(aspp_input)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    """ 3x3 conv rate=18 """
    y5 = Conv2D(256, 3, padding="same", use_bias=False, dilation_rate=18)(aspp_input)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = Concatenate()([y1, y2, y3, y4, y5])
    y = Conv2D(256, 1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y
    
    
def Deeplabv3_plus(img_h, img_w,
                   backbone,
                   low_feat_layer, high_feat_layer,
                   n_classes,
                   weights = "imagenet",
                   dropout = 0.3):
    """ Input """
    _input = Input((img_h, img_w, 3))
    
    """Encoder"""
    encoder = backbone(weights = weights, include_top = False, input_tensor = _input)
    # Take high level features from the backbone model
    image_features = encoder.get_layer(high_feat_layer).output
    #Put high level features through dilated average pooling
    x_a = ASPP(image_features)
    x_a = UpSampling2D((4,4), interpolation = "bilinear")(x_a)
    
    #Get low level features from backbone model
    x_b = encoder.get_layer(low_feat_layer).output
    x_b = Conv2D(filters=48, kernel_size=1, padding='same', use_bias=False)(x_b)
    x_b = BatchNormalization()(x_b)
    x_b = Activation('relu')(x_b)

    x = Concatenate()([x_a, x_b])
    x = SqueezeAndExcite(x)

    x = Conv2D(filters=256, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=256, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SqueezeAndExcite(x)

    x = UpSampling2D((4, 4), interpolation="bilinear")(x)
    x = Conv2D(n_classes, kernel_size =(1,1), padding = "same")(x)
    x = Activation("sigmoid")(x)

    model = Model(_input, x)
    return model
 

In [ ]:
IMG_H, IMG_W = IMAGE_SHAPE
BACKBONE = tf.keras.applications.ResNet50
LOW_FEAT_LAYER = "conv2_block2_out"
HIGH_FEAT_LAYER = "conv4_block6_out"
DO_TRAIN = False
#We can add addtional weights, but I will stick with the "imagenet" for now
# For now, I am trying to do "multilabel"
if STYLE=="multiclass":
    N_CLASSES = len(classes) # n_classses+background
else: 
    N_CLASSES = len(classes) # n_classses (binary so background is 0 in each channel)

MODEL_INSPECT = "summary"
SUB_NODEBUG_MODEL_WT_PATH = "../input/2021-5-30-trained-resnet50-256x256x3-multiclass/resnet50_256x256x3_multiclass"
WEIGHT_PATH = "../input/tf-keras-pretrained-model-weights/No Top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"

if DO_TRAIN:
    deeplabv3plus = Deeplabv3_plus(IMG_H, IMG_W,
                   BACKBONE,
                   LOW_FEAT_LAYER, HIGH_FEAT_LAYER,
                   n_classes = N_CLASSES, weights = WEIGHT_PATH)
    if MODEL_INSPECT=="plot":
        display(tf.keras.utils.plot_model(deeplabv3plus))
    elif MODEL_INSPECT=="summary":
        print(deeplabv3plus.summary())
else:
    deeplabv3plus = tf.keras.models.load_model(SUB_NODEBUG_MODEL_WT_PATH, compile=False)

# **TRANINING THE MODEL**

**Setting up evaluation metrics**

In [ ]:
from tensorflow.keras import backend as K
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    smooth = 0.0001
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_multilabel(y_true, y_pred, numLabels = N_CLASSES):
    dice=0
    for index in range(numLabels):
        dice += dice_coef(y_true[:,:,:,index], y_pred[:,:,:,index])
    return dice/numLabels

def dice_loss(y_true, y_pred):
    return (1 - dice_coef_multilabel(y_true, y_pred))

**Training**

In [ ]:
# Custom Callback To Include in Callbacks List At Training Time
from tensorflow.keras.metrics import Accuracy, Recall, Precision, MeanIoU
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        tf.keras.backend.clear_session()
        
def plot_history(_history, fold_num="1", metrics=("acc",)):
    """ TBD """
    fig = px.line(_history.history, 
                  x=range(len(_history.history["loss"])), 
                  y=["loss", "val_loss"],
                  labels={"value":"Loss (log-axis)", "x":"Epoch #"},
                  title=f"<b>FOLD {fold_num} MODEL - LOSS</b>", log_y=True
                  )
    fig.show()

    for _m in metrics:
        fig = px.line(_history.history, 
                      x=range(len(_history.history[_m])), 
                      y=[_m, f"val_{_m}"],
                      labels={"value":f"{_m} (log-axis)", "x":"Epoch #"},
                      title=f"<b>FOLD {fold_num} MODEL - {_m}</b>", log_y=True)
        fig.show()    

N_EPOCH = 10
if DO_TRAIN:
    OPTIMIZER = tf.keras.optimizers.Adam(0.0006666)
    if STYLE=="multiclass":
        LOSS = tf.keras.losses.SparseCategoricalCrossentropy()
    else:
        LOSS = tfa.losses.SigmoidFocalCrossEntropy()
        
    METRICS = ["acc"]
               

    _lr_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.75, 
                                                  patience=2, verbose=1, mode="min")
    _es_cb = tf.keras.callbacks.EarlyStopping(monitor="val_loss",  patience=4, 
                                              verbose=1, mode="min",
                                              restore_best_weights=True)
    _ckpt_cb = tf.keras.callbacks.ModelCheckpoint(f'./resnet50_{IMAGE_SHAPE[0]}x{IMAGE_SHAPE[1]}x3_{STYLE}', 
                                                  monitor='val_loss', mode="min",
                                                  save_best_only=True, options=save_locally)
    _gc_cb = GarbageCollectorCallback()
    CB_LIST = [_es_cb, _ckpt_cb, _lr_cb, _gc_cb]

    deeplabv3plus.compile(optimizer=OPTIMIZER, loss=dice_loss, metrics=[Accuracy(), dice_coef_multilabel])
    history = deeplabv3plus.fit(train_ds, validation_data=val_ds, epochs=N_EPOCH, callbacks=CB_LIST)
    

In [ ]:
def get_overlay(img, mask, _alpha=0.999, _beta=0.45, _gamma=0):
    
    # Normalize to be between 0-1 (float32)
    img = (img/img.max()).astype(np.float32)
    
    # Make mask RGB and float32
    if len(mask.shape)==2:
        mask_rgb = np.zeros_like(img, dtype=np.float32)
        mask_rgb[..., 2] = np.where(mask==3, 1.0, 0.0)
        mask_rgb[..., 1] = np.where(mask==2, 1.0, 0.0)
        mask_rgb[..., 0] = np.where(mask==1, 1.0, 0.0)
    else:
        mask_rgb=mask.astype(np.float32)
    
    # overlay
    seg_overlay = cv2.addWeighted(src1=img, alpha=_alpha, 
                                  src2=mask_rgb, beta=_beta, gamma=_gamma)
    return seg_overlay

def get_miss_overlay(gt_mask, pred_mask, _alpha=0.9, _beta=0.25, _gamma=0):
    
    # Make mask RGB and float32
    miss_rgb = np.zeros((*pred_mask.shape[:2],3), dtype=np.float32)
    if len(pred_mask.shape)==2:
        miss_rgb[..., 1] = np.where((gt_mask==pred_mask)&(gt_mask!=0), 0.8, 0.0)
        miss_rgb[..., 0] = np.where((gt_mask!=pred_mask), 0.8, 0.0)
    else:
        miss_rgb = np.where((gt_mask==pred_mask)&(gt_mask!=0.0), (0.0,0.8,0.0), (0.0,0.0,0.0))
        miss_rgb = np.where((gt_mask!=pred_mask), (0.8,0.0,0.0), miss_rgb)
        
    return miss_rgb

def plot_preds(img, pred_mask, gt_mask):
    gt_overlay = get_overlay(img, gt_mask)
    pred_overlay = get_overlay(img, pred_mask)
    #miss_overlay = get_miss_overlay(gt_mask, pred_mask)
    
    miss_overlay_lb = get_miss_overlay(gt_mask[...,0].squeeze().astype(np.float32), pred_mask[...,0].squeeze().astype(np.float32))
    miss_overlay_sb = get_miss_overlay(gt_mask[...,1].squeeze().astype(np.float32), pred_mask[...,1].squeeze().astype(np.float32))
    miss_overlay_st = get_miss_overlay(gt_mask[...,2].squeeze().astype(np.float32), pred_mask[...,2].squeeze().astype(np.float32))
    
    plt.figure(figsize=(27,17))
    
    for i, (_desc, _img) in enumerate(zip(["Prediction Mask", "Ground-Truth Mask", "Miss Mask Large Bowel", "Miss Mask Small Bowel", "Miss Mask Stomach"],
                                          [pred_overlay, gt_overlay, miss_overlay_lb, miss_overlay_sb, miss_overlay_st])):        
        plt.subplot(1,5,i+1)
        plt.imshow(_img)
        plt.title(f"{_desc} Image", fontweight="bold")        
        plt.axis(False)
        
        if i in [0,1]:
            handles = [Rectangle((0,0),1,1, color=_c) for _c in [(0.667,0.0,0.0), (0.0,0.667,0.0), (0.0,0.0,0.667)]]
            labels = ["Large Bowel Segmentation Map", "Small Bowel Segmentation Map", "Stomach Segmentation Map"]
            plt.legend(handles,labels)
        elif i in [2,3,4]:
            handles = [Rectangle((0,0),1,1, color=_c) for _c in [(0.0,0.8,0.0), (0.8,0.0,0.0), (0.0, 0.0, 0.0)]]
            labels = ["Agreement", "Disagreement", "Background"]
            plt.legend(handles,labels)
    plt.tight_layout()
    plt.show()

    """
    if DEBUG:
    for img_batch, mask_batch in val_ds.take(1):
        pred_batch = deeplabv3plus(img_batch)
        
        if STYLE=="multiclass":
            pred_batch = np.where(pred_batch>=0.5, 1.0, 0.0)
        else:
            pred_batch = np.argmax(pred_batch, axis=-1)

        img_batch = ((img_batch+1)*127.5).numpy().astype(np.int32)
        mask_batch = mask_batch.numpy().squeeze().astype(np.float32)
        break

    for _img, _pred, _mask in zip(img_batch, pred_batch, mask_batch):
        plot_preds(_img, _pred, _mask)"""
        

In [ ]:
def pred_2_rle(pred_arr, root_shape):
    
    # Get correct size pred array based on initial slice size
    lb_mask = cv2.resize(pred_arr[...,0], root_shape, interpolation=cv2.INTER_NEAREST)
    sb_mask = cv2.resize(pred_arr[...,1], root_shape, interpolation=cv2.INTER_NEAREST)
    st_mask = cv2.resize(pred_arr[...,2], root_shape, interpolation=cv2.INTER_NEAREST)
    # Get individual segmentation masks
    #lb_mask = np.where(pred_arr_lb==1,1,0)
    #sb_mask = np.where(pred_arr_sb==1,1,0)
    #st_mask = np.where(pred_arr_st==3,1,0)
    return rle_encode(lb_mask), rle_encode(sb_mask), rle_encode(st_mask)

N_TEST = int(np.ceil((len(ss_df)//3)/BATCH_SIZE))

print("\n\n\n\n\n\n ", "............NOW WE START PREDICTING.................", "\n\n\n\n\n\n\n")

# Loop over batches and get prediction
for i, img_batch in tqdm(enumerate(submission_ds), total=N_TEST):
    # Cleanup every so often
    if i%100==0:
        gc.collect(); gc.collect(); tf.keras.backend.clear_session(); gc.collect()    
    #print(img_batch.shape)
    
    # Get predictions
    pred_batch = deeplabv3plus(img_batch, training=False).numpy()
    pred_batch = np.where(pred_batch>=0.5, 1.0, 0.0)
    #pred_batch = tf.argmax(deeplabv3plus(img_batch, training=False), axis=-1).numpy()
    
    # Loop over prediction and determine submission dataframe index (3*individual-count because of reduced inference size)
    for j, _pred in enumerate(pred_batch):
        df_idx = 3*(i*BATCH_SIZE+j)
        pred_rles = pred_2_rle(_pred, (ss_df.iloc[df_idx]["slice_h"], ss_df.iloc[df_idx]["slice_w"]))
        # Loop over rles and assign the correct row of the submission dataframe
        for k,pred_rle in enumerate(pred_rles):
            ss_df.loc[df_idx+k, "predicted"] = pred_rle

In [ ]:
def fix_empty_slices(_row):
    identifier = _row["id"]
    slice_id = identifier.split("_",3)[3]
    if int(slice_id) in remove_seg_slices[_row["class"]]:
        _row["predicted"] = ""
    return _row

def is_isolated(_row):
    return (_row["predicted"]!="" and _row["prev_predicted"]=="" and _row["next_predicted"]=="")

def fix_nc_slices(_row):
    if _row["seg_isolated"]:
        _row["predicted"] = ""
    return _row

# No segmentation exists at these slices
remove_seg_slices = {
     "large_bowel": [1, 138, 139, 140, 141, 142, 143, 144],
    "small_bowel": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 138, 139, 140, 141, 142, 143, 144],
    "stomach": [1, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144],
}

# Heuristic processing
ss_df = ss_df.apply(fix_empty_slices, axis=1)

ss_df["prev_predicted"] = ss_df.shift(3, fill_value="")["predicted"]
ss_df["next_predicted"] = ss_df.shift(-3, fill_value="")["predicted"]
ss_df["seg_isolated"] = ss_df.apply(is_isolated, axis=1)
ss_df = ss_df.apply(fix_nc_slices, axis=1)

# Submit
ss_df = ss_df[["id", "class", "predicted"]]
ss_df.to_csv("submission.csv", index=False)
display(ss_df)